In [80]:
import pandas as pd
import numpy as np
import requests
import time
import io
import json

In [16]:
ruta = r"C:\Users\9 ----- SIG\Documents\EVALUACION DOCENTE\Análisis de sentimientos definitivo.xlsx"
df = pd.read_excel(ruta)
df.head(5)

,Profesor,Comentario,positivo,negativo,neutro,Punto crítico,Indicador
0,FERNANDO ROA,Decepcionado aque un doctor con tanto conocimi...,0.2,98.9,0.9,Decepcionado aque un doctor con tanto conocimi...,0.01
1,JUAN PERICO,Es un poco decepcionante ver a uno de los doce...,0.2,98.8,1.0,Es un poco decepcionante ver a uno de los doce...,0.01
2,ROCIO DE DIEGO,Es una de las peores docentes con las que me h...,0.3,98.6,1.1,Es una de las peores docentes con las que me h...,0.02
3,JULIAN PRADO,"El doctor Julián Prado muy mal profesor, no te...",0.3,98.6,1.2,"El doctor Julián Prado muy mal profesor, no te...",0.02
4,IVAN BUITRAGO,"Es un profesor muy prepotente, nos hablaba com...",0.2,98.6,1.2,"Es un profesor muy prepotente, nos hablaba com...",0.01


In [17]:
def clasificar_sentimiento(row):

    pos = row['positivo']
    neg = row['negativo']
    neu = row['neutro']

    # 1. Reglas principales
    if pos >= 0.70 and pos == max(pos, neg, neu):
        return "positivo"

    if neg >= 0.35 and neg == max(pos, neg, neu):
        return "negativo"

    if neu >= 0.50 and neu == max(pos, neg, neu):
        return "neutro"

    # 2. Si ninguna regla fuerte aplica → elegir el mayor valor
    maximo = max(pos, neg, neu)

    if maximo == pos:
        return "positivo"
    if maximo == neg:
        return "negativo"
    return "neutro"

In [18]:
# ---- CLASIFICAR CADA FILA ----
df["categoria"] = df.apply(clasificar_sentimiento, axis=1)

# ---- AGRUPAR POR PROFESOR ----
df_final = df.groupby("Profesor").agg(
    comentarios_positivos=("Comentario", lambda x: " | ".join(x[df.loc[x.index, "categoria"] == "positivo"])),
    comentarios_negativos=("Comentario", lambda x: " | ".join(x[df.loc[x.index, "categoria"] == "negativo"])),
    comentarios_neutros=("Comentario",  lambda x: " | ".join(x[df.loc[x.index, "categoria"] == "neutro"]))
).reset_index()

df_final

,Profesor,comentarios_positivos,comentarios_negativos,comentarios_neutros
0,ADIELA ZAPATA,excelente docente | excelente docente | Es imp...,,
1,ADOLFO PERALTA,Excelente doctor | Excelente docente | Excelen...,No es un buen tutor,Los porcentajes de la práctica bajan mucho a l...
2,ADOLFO PEREZ,Excelente docente | Excelente docente | Excele...,,
3,ADRIANA CEPEDA,Es un excelente profesora explica muy bien tie...,La profesora Adriana demuestra un buen dominio...,Casi no vimos clases con ella | Nunca he visto...
4,ADRIANA COLORADO,excelente maestra | excelente maestra | excele...,,
...,...,...,...,...
1290,YURY CASTRO,Excelente profesora | muy buena profesora y ex...,Esta docente nunca respondió dudas por medio d...,En una actividad de la segunda unidad (planeac...
1291,ZULMA MORENO,"Buena docente, tiene mucho conocimiento en su ...","Es la peor docente, no tiene objetividad, le g...",Sabe mucho pero es muy imparcial con las notas...
1292,ÁNGEL VILLANUEVA,Excelente profesor | Excelente profesor | Exce...,Deseo expresar mi inconformidad con el proceso...,Mejorar el vocabulario para referirse hacia lo...
1293,ÁNGELA GRANADOS,"Es una Excelente maestra, siempre busco el ben...","Me sentí muy estancada en el semestre, francam...",Sus sesiones fueron de una energía muy baja de...


In [24]:
ruta_2 = r"C:\Users\9 ----- SIG\Documents\EVALUACION DOCENTE\comentarios_preprocesados_profesor.xlsx"
df_final.to_excel(ruta_2, index = False)

### EJECUCICION DE API

In [84]:
# ================================
# CONFIGURACIÓN
# ================================
LMSTUDIO_URL = "http://10.2.1.99:1234/v1/chat/completions"
MODEL = "Qwen2.5-3B-Instruct-Q4_K_M"

# Archivo Excel preprocesado
ARCHIVO = r"C:\Users\9 ----- SIG\Documents\EVALUACION DOCENTE\comentarios_preprocesados_profesor.xlsx"

# Archivo de salida final
SALIDA = r"C:\Users\9 ----- SIG\Documents\EVALUACION DOCENTE\RESUMEN IA\resumen_docentes.csv"

In [85]:
# ================================
# Cargar el archivo
# ================================
df = pd.read_excel(ARCHIVO)

# Asegurar que están las columnas necesarias
columnas = ["Profesor", "comentarios_positivos", "comentarios_negativos", "comentarios_neutros"]
df = df[columnas]

resultados = []

In [86]:
# ==========================================
# 1. FUNCIÓN QUE PIDE Y PARSEA JSON
# ==========================================
def obtener_resumen_json(profesor, pos, neg, neu):
    # Prompt diseñado para forzar una estructura JSON estricta
    prompt = f"""
    Eres un motor de extracción de datos. NO converses.
    
    Tengo comentarios de un profesor:
    - POSITIVOS: {pos}
    - NEGATIVOS: {neg}
    - NEUTROS: {neu}

    Tu tarea: Generar un resumen muy breve (máx 15 palabras) para cada categoría.
    
    FORMATO OBLIGATORIO DE RESPUESTA (JSON):
    {{
        "positivo": "Aquí el resumen positivo",
        "negativo": "Aquí el resumen negativo",
        "neutro": "Aquí el resumen neutro"
    }}

    Si no hay información en una categoría, pon el valor null.
    RESPONDE SOLO CON EL JSON.
    """
    
    body = {
        "model": MODEL, # Asegúrate de tener definida tu variable MODEL
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.2,
        "max_tokens": 300
    }

    try:
        response = requests.post(LMSTUDIO_URL, json=body)
        raw_text = response.json()["choices"][0]["message"]["content"]
        
        # --- PARSEO ROBUSTO ---
        # A veces el modelo dice "Aquí está tu JSON: { ... }". 
        # Usamos Regex para buscar solo lo que está entre llaves { }
        match = re.search(r'\{.*\}', raw_text, re.DOTALL)
        
        if match:
            json_str = match.group(0)
            data = json.loads(json_str) # Convertimos texto a Diccionario Python
            return data
        else:
            return None # No se encontró JSON

    except Exception as e:
        print(f"Error procesando {profesor}: {e}")
        return None

In [ ]:
# ==========================================
# 2. PROCESAMIENTO LIMPIO
# ==========================================
filas_limpias = []

print(f"Procesando {len(df)} registros con estrategia JSON...")

for idx, row in df.iterrows():
    nombre_real = row["Profesor"]
    
    # Convertir a string y limpiar nulos de entrada
    p = str(row.get("comentarios_positivos", ""))
    n = str(row.get("comentarios_negativos", ""))
    ne = str(row.get("comentarios_neutros", ""))
    
    # Llamamos a la función
    resultado_json = obtener_resumen_json(nombre_real, p, n, ne)
    
    # Valores por defecto si falla
    res_pos = "Sin datos"
    res_neg = "Sin datos"
    res_neu = "Sin datos"
    
    if resultado_json:
        # Extraemos usando las claves. .get() evita errores si falta una clave
        # Si el valor es None (null en json), ponemos "Sin datos"
        res_pos = resultado_json.get("positivo") or "Sin datos"
        res_neg = resultado_json.get("negativo") or "Sin datos"
        res_neu = resultado_json.get("neutro") or "Sin datos"
    
    filas_limpias.append({
        "Profesor": nombre_real,
        "Resumen_Positivo": res_pos,
        "Resumen_Negativo": res_neg,
        "Resumen_Neutro": res_neu
    })
    
    # Barra de progreso simple
    if idx % 5 == 0:
        print(f"Fila {idx} completada...")
        
    time.sleep(0.1)

Procesando 1295 registros con estrategia JSON...
Fila 0 completada...
Fila 5 completada...
Fila 10 completada...
Fila 15 completada...
Fila 20 completada...


In [ ]:
df_final = pd.DataFrame(filas_limpias)
df_final

In [ ]:
# GUARDAR

df_final.to_excel(r"C:\Users\9 ----- SIG\Documents\EVALUACION DOCENTE\RESUMEN IA\resumen_docente_IA.xlsx", index = False)